In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
df = pd.read_csv("../Data/all_objects.csv")

In [3]:
sl = df[df['OBJECT_ID'] == '1988-102B']

In [7]:
from skyfield.api import EarthSatellite, load, wgs84
from datetime import datetime

#Добавляем колонку с высотами объектов

ts = load.timescale()
planets = load('de421.bsp')
earth = planets['earth']

def calculate_altitude(tle_line1, tle_line2):
    satellite = EarthSatellite(tle_line1, tle_line2, 'Satellite', ts)
    t = ts.now()
    geocentric = satellite.at(t)
    subpoint = wgs84.subpoint(geocentric)
    return subpoint.elevation.km

sl['altitude_km'] = sl.apply(lambda row: calculate_altitude(row['TLE_LINE1'], row['TLE_LINE2']), axis=1)

sl

C:\Users\snoga\AppData\Local\Temp\ipykernel_1296\2308469628.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sl['altitude_km'] = sl.apply(lambda row: calculate_altitude(row['TLE_LINE1'], row['TLE_LINE2']), axis=1)


,CCSDS_OMM_VERS,COMMENT,CREATION_DATE,ORIGINATOR,OBJECT_NAME,OBJECT_ID,CENTER_NAME,REF_FRAME,TIME_SYSTEM,MEAN_ELEMENT_THEORY,...,COUNTRY_CODE,LAUNCH_DATE,SITE,DECAY_DATE,FILE,GP_ID,TLE_LINE0,TLE_LINE1,TLE_LINE2,altitude_km
18073,3.0,GENERATED VIA SPACE-TRACK.ORG API,2025-03-12T02:20:35,18 SPCS,SL-16 R/B,1988-102B,EARTH,TEME,UTC,SGP4,...,CIS,1988-11-23,TTMTR,NaN,4666552,282863660,0 SL-16 R/B,1 19650U 88102B 25070.76213488 -.00000163 0...,2 19650 70.9997 195.0871 0015391 211.3506 148...,831.603442


In [9]:
sl.columns

Index(['CCSDS_OMM_VERS', 'COMMENT', 'CREATION_DATE', 'ORIGINATOR',
       'OBJECT_NAME', 'OBJECT_ID', 'CENTER_NAME', 'REF_FRAME', 'TIME_SYSTEM',
       'MEAN_ELEMENT_THEORY', 'EPOCH', 'MEAN_MOTION', 'ECCENTRICITY',
       'INCLINATION', 'RA_OF_ASC_NODE', 'ARG_OF_PERICENTER', 'MEAN_ANOMALY',
       'EPHEMERIS_TYPE', 'CLASSIFICATION_TYPE', 'NORAD_CAT_ID',
       'ELEMENT_SET_NO', 'REV_AT_EPOCH', 'BSTAR', 'MEAN_MOTION_DOT',
       'MEAN_MOTION_DDOT', 'SEMIMAJOR_AXIS', 'PERIOD', 'APOAPSIS', 'PERIAPSIS',
       'OBJECT_TYPE', 'RCS_SIZE', 'COUNTRY_CODE', 'LAUNCH_DATE', 'SITE',
       'DECAY_DATE', 'FILE', 'GP_ID', 'TLE_LINE0', 'TLE_LINE1', 'TLE_LINE2',
       'altitude_km'],
      dtype='object')

In [11]:
sl['ECCENTRICITY']

18073    0.001539
Name: ECCENTRICITY, dtype: float64

In [13]:
sl['INCLINATION']

18073    70.9997
Name: INCLINATION, dtype: float64

In [15]:
#Долгота восходящего узла
sl['RA_OF_ASC_NODE']

18073    195.0871
Name: RA_OF_ASC_NODE, dtype: float64

In [17]:
sl['ARG_OF_PERICENTER']

18073    211.3506
Name: ARG_OF_PERICENTER, dtype: float64

In [19]:
sl['MEAN_ANOMALY']


18073    148.6701
Name: MEAN_ANOMALY, dtype: float64

In [21]:
sl['BSTAR']

18073   -0.000058
Name: BSTAR, dtype: float64

In [23]:
sl['SEMIMAJOR_AXIS']

18073    7216.777
Name: SEMIMAJOR_AXIS, dtype: float64

In [25]:
sl['APOAPSIS']

18073    849.75
Name: APOAPSIS, dtype: float64

In [27]:
sl['PERIAPSIS']

18073    827.535
Name: PERIAPSIS, dtype: float64

In [19]:
sl['TLE_LINE1'].values[0]

'1 19650U 88102B   25070.76213488 -.00000163  00000-0 -58357-4 0  9997'

In [12]:
from skyfield.api import load, EarthSatellite
import numpy as np
from datetime import timedelta

tle = [
    "1 25544U 98067A   20232.51736111  .00000123  00000-0  12345-4 0  9999",
    "2 25544  51.6448 208.9163 0004018 315.5941 218.7776 15.49165421234567"
]
satellite = EarthSatellite(sl['TLE_LINE1'].values[0], sl['TLE_LINE2'].values[0], 'SL-16 R/B', load.timescale())

# Параметры двигателя
thrust = 1.2e-3  
isp = 2300
g0 = 9.81 

# Масса аппарата (предположим, что начальная масса 1000 кг)
initial_mass = 1000.0  # кг

# Целевая высота
target_altitude = 100000  # 100 км в метрах

# Загрузка данных о Земле
ts = load.timescale()
eph = load('de421.bsp')
earth = eph['earth']

# Функция для расчета высоты над поверхностью Земли
def calculate_altitude(satellite, t):
    geocentric = satellite.at(t)
    subpoint = geocentric.subpoint()
    return subpoint.elevation.m  # Высота в метрах

# Начальное время
start_time = ts.now()

# Шаг времени для симуляции (например, 1 минута)
time_step = timedelta(seconds=60*60*24*100)  # 60 секунд

# Симуляция до достижения целевой высоты
current_time = start_time
altitude = calculate_altitude(satellite, current_time)
while altitude > target_altitude:
    # Увеличиваем время на шаг
    current_time = ts.utc(current_time.utc_datetime() + time_step)
    altitude = calculate_altitude(satellite, current_time)
    if altitude <= target_altitude:
        break

# Время спуска
descent_time = (current_time - start_time)  # Время в днях

# Расчет характеристической скорости (ΔV)
delta_v = isp * g0 * np.log(initial_mass / (initial_mass - (thrust * descent_time / (isp * g0))))

# Масса топлива
fuel_mass = thrust * descent_time / (isp * g0)

# Вывод результатов
print(f"Время спуска: {descent_time} дней")
print(f"Затраты характеристической скорости (ΔV): {delta_v} м/с")
print(f"Масса топлива: {fuel_mass} кг")

Время спуска: 535300.0 дней
Затраты характеристической скорости (ΔV): 0.6423691440429403 м/с
Масса топлива: 0.028469618401808266 кг


In [23]:
from skyfield.api import load, EarthSatellite
import numpy as np
from datetime import timedelta

# Загрузка данных TLE
tle = [
    "1 25544U 98067A   20232.51736111  .00000123  00000-0  12345-4 0  9999",
    "2 25544  51.6448 208.9163 0004018 315.5941 218.7776 15.49165421234567"
]
satellite = EarthSatellite(sl['TLE_LINE1'].values[0], sl['TLE_LINE2'].values[0], 'SL-16 R/B', load.timescale())

# Параметры двигателя
thrust = 1.2e-3  # Тяга в Ньютонах
isp = 2300  # Удельный импульс в секундах
g0 = 9.81  # Стандартное ускорение свободного падения

# Масса аппарата (предположим, что начальная масса 1000 кг)
initial_mass = 8900.0  # кг

# Целевая высота
target_altitude = 100000  # 100 км в метрах

# Загрузка данных о Земле
ts = load.timescale()
eph = load('de421.bsp')
earth = eph['earth']

# Функция для расчета высоты над поверхностью Земли
def calculate_altitude(satellite, t):
    geocentric = satellite.at(t)
    subpoint = geocentric.subpoint()
    return subpoint.elevation.m  # Высота в метрах

# Начальное время
start_time = ts.now()

# Шаг времени для симуляции (например, 1 минута)
time_step = timedelta(seconds=60*60*24*100)  # 60 секунд

# Начальные условия
current_time = start_time
altitude = calculate_altitude(satellite, current_time)
mass = initial_mass
delta_v = 0.0  # Накопленная характеристическая скорость

# Симуляция до достижения целевой высоты
while altitude > target_altitude:
    # Расчет ускорения от двигателя
    acceleration = thrust / mass  # Ускорение в м/с²

    # Обновление скорости (ΔV = a * Δt)
    delta_v += acceleration * time_step.total_seconds()

    # Обновление массы (расход топлива)
    mass_flow_rate = thrust / (isp * g0)  # Расход топлива в кг/с
    mass -= mass_flow_rate * time_step.total_seconds()

    # Обновление времени
    current_time = ts.utc(current_time.utc_datetime() + time_step)

    # Расчет новой высоты
    altitude = calculate_altitude(satellite, current_time)

    # Проверка достижения целевой высоты
    if altitude <= target_altitude:
        break

# Время спуска (в секундах)
descent_time = (current_time - start_time) * 86400  # Преобразуем дни в секунды

# Масса топлива
fuel_mass = initial_mass - mass

# Вывод результатов
print(f"Время спуска: {descent_time} секунд ({descent_time / 86400} дней)")
print(f"Затраты характеристической скорости (ΔV): {delta_v} м/с")
print(f"Масса топлива: {fuel_mass} кг")

Время спуска: 46241280000.0 секунд (535200.0 дней)
Затраты характеристической скорости (ΔV): 7297.023090259428 м/с
Масса топлива: 2459.3155165517237 кг


<Time tt=2460747.022338154>

In [13]:
from skyfield.api import load, EarthSatellite, wgs84
import numpy as np
from datetime import timedelta, datetime

# Начальные данные для Orion 50 XL
initial_position = [3038.4, 412.0, -7069.3]  # Положение в км
initial_velocity = [-3.71, -5.84, 0.98]  # Скорость в км/с

# Преобразуем в метры и метры/секунду
initial_position = np.array(initial_position) * 1000  # Переводим в метры
initial_velocity = np.array(initial_velocity) * 1000  # Переводим в м/с

# Параметры двигателя
thrust = 1.2e-3  # Тяга в Ньютонах
isp = 2300  # Удельный импульс в секундах
g0 = 9.81  # Стандартное ускорение свободного падения

# Масса аппарата (предположим, что начальная масса 1000 кг)
initial_mass = 400.0  # кг

# Целевая высота
target_altitude = 100000  # 100 км в метрах

# Загрузка данных о Земле
ts = load.timescale()
eph = load('de421.bsp')
earth = eph['earth']

# Создание объекта EarthSatellite
satellite = EarthSatellite(
    "1 00000U 00000A   00000.00000000  .00000000  00000-0  00000-0 0  9999",
    "2 00000  51.6448 208.9163 0004018 315.5941 218.7776 15.49165421234567",
    'Orion 50 XL', ts
)

# Начальное время (текущее время)
start_time = ts.now()

# Шаг времени для симуляции (например, 1 минута)
time_step = timedelta(seconds=60*24*60)  # 60 секунд

# Начальные условия
current_time = start_time
mass = initial_mass
delta_v = 0.0  # Накопленная характеристическая скорость

# Симуляция до достижения целевой высоты
while True:
    # Расчет ускорения от двигателя
    acceleration = thrust / mass  # Ускорение в м/с²

    # Обновление скорости (ΔV = a * Δt)
    delta_v += acceleration * time_step.total_seconds()

    # Обновление массы (расход топлива)
    mass_flow_rate = thrust / (isp * g0)  # Расход топлива в кг/с
    mass -= mass_flow_rate * time_step.total_seconds()

    # Обновление времени
    current_time = ts.utc(current_time.utc_datetime() + time_step)

    # Расчет новой высоты
    geocentric = satellite.at(current_time)
    subpoint = geocentric.subpoint()
    altitude = subpoint.elevation.m  # Высота в метрах

    # Проверка достижения целевой высоты
    if altitude <= target_altitude:
        break

    # Проверка на выход за пределы допустимого диапазона времени
    if current_time.utc_datetime().year > 2100 or current_time.utc_datetime().year < 1900:
        print("Ошибка: Время вышло за пределы допустимого диапазона.")
        break

# Время спуска (в секундах)
descent_time = (current_time - start_time) * 86400  # Преобразуем дни в секунды

# Масса топлива
fuel_mass = initial_mass - mass

# Вывод результатов
print(f"Время спуска: {descent_time} секунд ({descent_time / 86400} дней)")
print(f"Затраты характеристической скорости (ΔV): {delta_v} м/с")
print(f"Масса топлива: {fuel_mass} кг")

Ошибка: Время вышло за пределы допустимого диапазона.
Время спуска: 2392243200.0 секунд (27688.0 дней)
Затраты характеристической скорости (ΔV): 8637.864093770402 м/с
Масса топлива: 127.23005983268422 кг


In [ ]:
satellite = EarthSatellite(sl['TLE_LINE1'].values[0], sl['TLE_LINE2'].values[0], 'SL-16 R/B', load.timescale())